In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages #reducer function

In [3]:
llm = init_chat_model("google_genai:gemini-2.5-flash-lite", temperature=0)

#Testing API Key
llm.invoke("How many o's in Google?")

AIMessage(content='There are **two** "o"s in the word "Google".', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019c2988-ca4d-7d03-981c-6bf644db1377-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 9, 'output_tokens': 15, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}})

In [4]:
class State(TypedDict):
    #messages: list -> For simple 1 message preservation 
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    ai_msg = llm.invoke(state["messages"])
    return {"messages": [ai_msg]}

In [5]:
builder = StateGraph(State)

builder.add_node("chatbotNode", chatbot)

builder.add_edge(START, "chatbotNode")
builder.add_edge("chatbotNode", END)

chatGraph = builder.compile()

In [6]:
response = chatGraph.invoke({"messages": [HumanMessage(content="What is the capital of India?")]})

response["messages"][-1].content

'The capital of India is **New Delhi**.'